In [2]:
import pandas as pd
import numpy as np
import re
import os

In [8]:
SURVEY_PATH= os.path.join("..", "data", "final_survey_data.csv")
df_raw = pd.read_csv(SURVEY_PATH)

### Filter emails

In [10]:
def retrieve_emails(df_list):
    """
    Find and export a list of emails. 
    """
    if len(df_list) == 1:
        emails = df_list[0]['Q25.1'][2:].to_list()
    elif len(df_list) > 1:
        emails = []
        for df in df_list:
            l = df['Q25.1'][2:].to_list()
            emails += l
    filtered_emails = [email for email in emails if not pd.isnull(email)==True]
    
    output_path = os.path.join("..", "outputs", "emails.txt")
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as output:
        output.write(str(filtered_emails))

retrieve_emails([df_raw])

### Data cleaning

In [11]:
def filter_data(df):
    """
    Filters out unfinished survey data, emails, and unnecessary columns.
    """
    column_descriptions = df[:2]
    df_finished = df[df['Finished']=='True']
    columns_to_remove = ['Finished', 'StartDate', 'EndDate', 'Status', 'Progress', 'DistributionChannel', 'UserLanguage', 'Q25.1']
    column_descriptions.drop(columns=columns_to_remove, inplace=True)
    df_clean = df_finished.drop(columns=columns_to_remove)

    return column_descriptions, df_clean

In [12]:
column_descriptions, df_all = filter_data(df_raw)
df_all.to_csv("../outputs/filtered_survey_data.csv")

/var/folders/qc/8fy08h5s33n4rrmz09_8bv680000gn/T/ipykernel_84043/185804197.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column_descriptions.drop(columns=columns_to_remove, inplace=True)


### Compile songs

In [13]:
strategy_matrix_question_number = { 
       "entertainment": "20",
        "revival": "41",
        "strong_sensation": "45",
        "diversion": "48",
        "discharge": "51",
        "mental_work": "54",
        "solace": "57"
    }


def extract_spotify_track_id(df):
    """
    Extract the Spotify Track ID from a Spotify link.
    """
    pattern = r"https?:\/\/(?:open\.)?spotify\.com\/track\/([a-zA-Z0-9]+)"
    track_ids = []

    for _, row in df.iterrows():
        url = row.get("url")
        
        if isinstance(url, str):
            match = re.search(pattern, url)
            track_ids.append(match.group(1) if match else None)

        else:
            track_ids.append(None)

    return pd.Series(track_ids, index=df.index)

def generate_clean_spotify_url(df):
    """
    Remove personal information from Spotify links
    """
    clean_urls = []
    for _, row in df.iterrows():
        url = row.get("url")
        track_id = row.get("track_id")

        if isinstance(url, str) and "spotify.com/track" in url and pd.notnull(track_id):
            clean_urls.append(f"https://open.spotify.com/track/{track_id}")
        else:
            clean_urls.append(url)
    
    df["url"] = clean_urls
    return df

def extract_song_columns(df, question_number, song_number, verbose=True):
    wanted_cols = [f"Q{question_number}#1_{song_number}_{j}" for j in (1, 2, 3, 4) if f"Q{question_number}#1_{song_number}_{j}" in df.columns]
    wanted_cols += [f"Q{question_number}#2_{song_number}", f"Q{question_number}#3_{song_number}"]
    return wanted_cols 

def compile_songs(df, question_number):
    """
    Compile songs for a given strategy
    """
    songs = []
    columns= ["song_name", "artist", "url", "description", "listen_in_non_stress_regulating_setting", "lyrics_is_a_important_part", "response_id"]

    for i in range(1, 6):
        cols = extract_song_columns(df, question_number, i)
        mask = df[cols].notna().any(axis=1)
        cols.append("ResponseId")
        subset = df.loc[mask, cols].copy()
        subset.columns = columns
        subset = subset[subset["song_name"].astype(str).str.len() > 1]
        songs.append(subset)

    songs = pd.concat(songs, axis=0, ignore_index=True)
    return songs


def compile_songs_all_strategies(df, export=False, export_name=''):
    """
    Compile all songs from all strategies, along with song annotations.
    """
    song_lists_by_strategy = []
    df_base = df.copy()

    for strategy, qnum in strategy_matrix_question_number.items():
        songs_by_strategy = compile_songs(df_base, qnum)
        songs_by_strategy["strategy"] = strategy
        song_lists_by_strategy.append(songs_by_strategy)

    df_compiled = pd.concat(song_lists_by_strategy, ignore_index=True)
    df_compiled['track_id'] = extract_spotify_track_id(df_compiled)
    df_compiled = generate_clean_spotify_url(df_compiled)

    df_compiled = df_compiled[["song_name", "artist", "url", "track_id", "description", "listen_in_non_stress_regulating_setting", "lyrics_is_a_important_part", "response_id", "strategy"]]

    if export:
        df_compiled.to_csv(export_name, index=False)

    return df_compiled

In [15]:
df_compiled_songs = compile_songs_all_strategies(df_all, export=False, export_name=output_path)

In [22]:
# df_compiled_songs_1 = pd.read_csv("../../src/all_strategies_songs_clean.csv")
# df_compiled_songs_2 = pd.read_csv("../../src/second_batch_clean.csv")
# df_compiled_songs_3 = pd.read_csv("../../src/third_batch_songs.csv")
# df_compiled_songs = pd.concat([df_compiled_songs_1, df_compiled_songs_2, df_compiled_songs_3])

### Process Duplicates

In [36]:
# If there are more than one url link for the same (song, artist) pair, keep the first occurence.
df_compiled_songs_unique= df_compiled_songs.sort_values('url').drop_duplicates(subset=['song_name', 'artist'], keep='first')

In [37]:
def handle_strategies_for_dupes(df):
    """
    The strategies for duplicated tracks contain all unique strategies for the same track,
    along with their counts as tuples like ('revival', 3).
    """
    df = df.copy()
    dup_mask_url = df['url'].duplicated(keep=False)

    for url, group in df[dup_mask_url].groupby('url'):
        strategy_counts = group['strategy'].value_counts().items()
        strategy_summary = sorted(strategy_counts)
        df.loc[df['url'] == url, 'strategy'] = [strategy_summary] * len(group)

    return df

In [38]:
df_compiled_songs_counted_strategies = handle_strategies_for_dupes(df_compiled_songs_unique)

In [42]:
df_compiled_songs_counted_strategies = handle_strategies_for_dupes(df_compiled_songs_unique)

output_path = os.path.join("..", "outputs", "annotated_compiled_songs_with_strategy_counts.csv")
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_compiled_songs_counted_strategies.to_csv(output_path)

In [41]:
df_compiled_songs_counted_strategies.head()

,song_name,artist,url,track_id,description,listen_in_non_stress_regulating_setting,lyrics_is_a_important_part,response_id,strategy
158,デリヘル呼んだら君が来た,メガテラ・ゼロ,https://music.youtube.com/watch?v=BIIiCRQhnWw&...,NaN,NaN,Yes,No,R_8eLKaSUgqXjGFkl,diversion
192,私のアール,メガテラ・ゼロ,https://music.youtube.com/watch?v=Tsl-2_QMhqE&...,NaN,NaN,Yes,Yes,R_8eLKaSUgqXjGFkl,discharge
220,灯火の命題,メガテラ・ゼロ,https://music.youtube.com/watch?v=pUzHLc_5cb0&...,NaN,NaN,Yes,Yes,R_8eLKaSUgqXjGFkl,discharge
193,Swing swing,The all-american rejects,https://open.spotify.com/track/003FTlCpBTM4eSq...,003FTlCpBTM4eSqYSWPv4H,NaN,Yes,No,R_4lLXShteMUjWgMW,revival
111,Mr brightside,The killers,https://open.spotify.com/track/003vvx7Niy0yvhv...,003vvx7Niy0yvhvHt4a68B,NaN,Yes,No,R_4lLXShteMUjWgMW,"[(entertainment, 1), (revival, 1)]"
